In [ ]:
!pip install eli5

In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [ ]:
cd "/content/drive/My Drive/Colab Notebooks/Machine-Learning"

/content/drive/My Drive/Colab Notebooks/Machine-Learning


In [ ]:
df=pd.read_csv("data/men_shoes.csv", low_memory=False)

In [ ]:
def run_model (feats,model=DecisionTreeRegressor(max_depth=5)):
  X=df[feats].values
  Y=df['prices_amountmin'].values

  scores=cross_val_score (model, X,Y,scoring='neg_mean_absolute_error')
  return np.mean(scores),np.std(scores)

In [ ]:
df['brand_cat']=df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])


(-58.133398968282776, 4.206122611474276)

In [ ]:
model=RandomForestRegressor(max_depth=5, n_estimators=100,random_state=0)
run_model(['brand_cat'],model)

(-57.31783843165656, 4.181246596160967)

In [ ]:
df.head()

In [ ]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [ ]:
str_dict='[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'
literal_eval(str_dict)

[{'key': 'Gender', 'value': ['Men']},
 {'key': 'Shoe Size', 'value': ['M']},
 {'key': 'Shoe Category', 'value': ["Men's Shoes"]},
 {'key': 'Color', 'value': ['Multicolor']},
 {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']},
 {'key': 'Brand', 'value': ['Josmo']}]

In [ ]:
def parse_features(x):
  output_dict={}
  if str(x)=='nan':return output_dict

  features= literal_eval(x.replace('\\"','"'))
  for item in features :
    key=item['key'].lower().strip()
    value=item['value'][0].lower().strip()

    output_dict[key]=value

  return output_dict
df['features_parsed']=df['features'].map(parse_features)

In [ ]:
keys=set()

df['features_parsed'].map(lambda x: keys.update(x.keys()))
len(keys)

476

In [ ]:
def get_name_feat(key):
  return 'feat_'+key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)]=df.features_parsed.map(lambda feats:feats[key] if key in feats else np.nan )
  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [ ]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_battery voltage', 'feat_lens width', 'feat_batteries required?',
       'feat_material', 'feat_rx-able', 'feat_terrain',
       'feat_combined shipping', 'feat_kids backpacks', 'feat_chain length',
       'feat_fits most screen size'],
      dtype='object', length=526)

In [ ]:
keys_stat={}
for key in keys:
  keys_stat[key]=df[False==df[get_name_feat(key)].isnull()].shape[0]/df.shape[0]*100

In [ ]:
{k:v for k,v in keys_stat.items() if v>30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [ ]:
df['feat_brand_cat']=df['feat_brand'].factorize()[0]
df['feat_color_cat']=df['feat_color'].factorize()[0]
df['feat_gender_cat']=df['feat_gender'].factorize()[0]
df['feat_material_cat']=df['feat_material'].factorize()[0]
df['feat_manufacturer part number_cat']=df['feat_manufacturer part number'].factorize()[0]

df['feat_sport_cat']=df['feat_sport'].factorize()[0]
df['feat_style_cat']=df['feat_style'].factorize()[0]


In [ ]:
df.brand=df.brand.map(lambda x: str(x).lower())
df[df.brand==df.feat_brand].shape

feats=['']

In [ ]:
def lbs_to_g(value):
  value_int=int(value*453.6)
  return value_int

def pound_to_g(value):
  value_int=int(value*453.6)
  return value_int

def ounce_to_g(value):
  value_int=int(value*28.35)
  return value_int

def kg_to_g(value):
  value_int=int(value*1000)
  return value_int

In [ ]:
df.weight=df.weight.fillna(0)

In [ ]:
for i in range(0,df.weight.size):
  if df.weight[i]!=0:
    d_split=df.weight.str.split(pat=' ')[i] 
    if d_split[1].lower() != 'g':
      if d_split[1].lower() == 'lbs':df.weight[i]= '{} g'.format(lbs_to_g(float(d_split[0])))
      elif d_split[1].lower() == 'pounds':df.weight[i]= '{} g'.format(pound_to_g(float(d_split[0])))
      elif d_split[1].lower() == 'ounces':df.weight[i]= '{} g'.format(ounce_to_g(float(d_split[0])))
      elif d_split[1].lower() == 'kg': df.weight[i]='{} g'.format(kg_to_g(float(d_split[0])))

In [ ]:
df.weight

In [ ]:
df['weight_cat']=df['weight'].factorize()[0]


In [ ]:
feats=['brand_cat','feat_material_cat','feat_brand_cat','weight_cat']
#feats+=feats_cat

model=RandomForestRegressor(max_depth=5,n_estimators=100)
run_model(feats,model)

In [287]:
X=df[feats ].values
Y=df['prices_amountmin'].values

m=RandomForestRegressor(max_depth=5,n_estimators=100, random_state=0)
m.fit(X,Y)

perm =PermutationImportance(m,random_state=1).fit(X,Y);
eli5.show_weights(perm,feature_names=feats)

Weight,Feature
0.2824 ± 0.0091,brand_cat
0.1117 ± 0.0133,feat_material_cat
0.0362 ± 0.0023,feat_brand_cat
0.0056 ± 0.0005,weight_cat
